In [ ]:
import tweepy

consumer_key = 'aNTJRrnNf2PI9JXP7AvpkTB4M'
consumer_secret = 'bPu9Rcwuyn3ps8JNN6UqzEZYfmWo7RbboC8YcNaCLQ3NicD0Ci'
access_token = 'AAAAAAAAAAAAAAAAAAAAAGVMKQEAAAAAz0p5ZQfE7AVCOmU6XhOU%2BrMeZdA%3Dg6T4oRRspibrHZuQW7mr5RTFQUoQ7BRXxkSYQe1LuJexZ5yM5X'
access_secret = 'YOUR ACCESS SECRET'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)

# Introduction and import of modules

# Description of data sources and import of data

# Data Wrangling

##  Assesment

### Data tidyness

### Data quality

## Cleaning

### Issues and solutions

# Analysis

## Data transformation and augmentation

## Prelimenary analysis

## Statistical Analysis

# Conclusions